# Sesiunea 17 din modulul 16-17_Pandas – Pipeline-uri de curățare pe serii temporale
_Notebook de exerciții (fără soluții) din seria 16-17_Pandas, continuând mini-inventarul exportat la sesiunea 16 și pregătind datele pentru vizualizările Matplotlib._


### Exercițiul 1 – Set de vânzări cu index temporal
Extinde contextul comercial început anterior și construiește un `DataFrame` cu date zilnice generate prin `pd.date_range` (de exemplu, `2023-01-01` → `2023-03-31`). Include coloanele `Data`, `Oraș`, `Produs` și `Vânzări`, setează data drept index (`DatetimeIndex`), verifică tipurile pentru a confirma `datetime64[ns]` și inspectează primele și ultimele cinci rânduri.


In [1]:
import pandas as pd
import random

cities = [
    "New York", "Philadelphia", "Boston", "Washington D.C.",
    "Baltimore", "Pittsburgh", "Cleveland", "Buffalo",
    "Hartford", "Providence"]

products = [
    "Laptop", "Desktop PC", "Smartphone", "Tablet",
    "Smartwatch", "Headphones", "Bluetooth Speaker",
    "Keyboard", "Mouse", "Monitor", "Webcam", "Printer"]

dates = pd.date_range(start="2023-01-01", freq="ME", end="2023-03-31")
data = {
    "City": [random.choice(cities) for _ in range(len(dates))],
    "Product": [random.choice(products) for _ in range(len(dates))],
    "Sales": [random.randint(1, 25) for _ in range(len(dates))]
}

df = pd.DataFrame(data, index=dates)

df

,City,Product,Sales
2023-01-31,Boston,Smartphone,20
2023-02-28,Cleveland,Laptop,3
2023-03-31,Buffalo,Keyboard,2


### Exercițiul 2 – Zoom pe o săptămână relevantă
Continuând seria temporală, filtrează o săptămână completă folosind atât un slice pe index (`df.loc['YYYY-MM-DD':'YYYY-MM-DD']`), cât și o condiție booleană echivalentă. Raportează forma subsetului, rulează `describe()` pentru intervalul selectat și confirmă ordinea cronologică, precum și absența duplicatelor de dată.


In [2]:
temp = df.loc["2023-02-28":"2023-03-31"]

temp.describe()

,Sales
count,2.000000
mean,2.500000
std,0.707107
min,2.000000
25%,2.250000
50%,2.500000
75%,2.750000
max,3.000000


In [3]:
temp

,City,Product,Sales
2023-02-28,Cleveland,Laptop,3
2023-03-31,Buffalo,Keyboard,2


### Exercițiul 3 – Pivot pentru comparații între orașe și produse
Pregătește analiza strategică generând un pivot cu `values='Vânzări'`, `index='Oraș'`, `columns='Produs'` și `aggfunc='mean'`. Sortează rezultatul descrescător după media totală pe oraș, adaugă totaluri (`margins=True`), formatează numeric cu două zecimale și compară-l textual cu o soluție `groupby` echivalentă.


In [4]:
pivot = pd.pivot_table(df, values="Sales", index="City", columns="Product", aggfunc="mean", margins=True)

pivot

Product,Keyboard,Laptop,Smartphone,All
City,,,,
Boston,NaN,NaN,20.0,20.000000
Buffalo,2.0,NaN,NaN,2.000000
Cleveland,NaN,3.0,NaN,3.000000
All,2.0,3.0,20.0,8.333333


### Exercițiul 4 – Strategie pentru valori lipsă
Simulează erori de raportare introducând deliberat `NaN` în coloana `Vânzări`, apoi cuantifică lipsurile cu `df.isnull().sum()` și procentul aferent pe coloană. Testează `dropna()` versus `fillna()` (de exemplu, folosind media sau mediana per pereche `Oraș`–`Produs`), notează într-un comentariu avantajele fiecărei abordări și păstrează varianta preferată pentru pașii următori.


In [5]:
pivot.isnull().sum()

Product
Keyboard      2
Laptop        2
Smartphone    2
All           0
dtype: int64

In [6]:
pivot.dropna()

Product,Keyboard,Laptop,Smartphone,All
City,,,,
All,2.0,3.0,20.0,8.333333


In [7]:
pivot.fillna("None")

Product,Keyboard,Laptop,Smartphone,All
City,,,,
Boston,None,None,20.0,20.000000
Buffalo,2.0,None,None,2.000000
Cleveland,None,3.0,None,3.000000
All,2.0,3.0,20.0,8.333333


### Exercițiul 5 – Funcție reutilizabilă de curățare
Împachetează logica într-o funcție `clean_data(df)` care elimină rândurile invalide, convertește tipurile (asigurându-se că `Vânzări` este `float`) și domolește outlierii cu un `clip` pe percentile. Aplică funcția într-un pipeline Pandas (`df.pipe(clean_data)`), compară dimensiunile și valorile lipsă înainte/după și adaugă un docstring plus validări minime pentru reutilizare în modulele de vizualizare.


In [8]:
from pandas import DataFrame

def clean_data(df: DataFrame):
    df["Sales"] = [float(x) for x in df["Sales"]]
    return df.dropna()

new_df = clean_data(df)

new_df

,City,Product,Sales
2023-01-31,Boston,Smartphone,20.0
2023-02-28,Cleveland,Laptop,3.0
2023-03-31,Buffalo,Keyboard,2.0
